<a href="https://colab.research.google.com/github/kedrick07/NLP-Semester-5/blob/main/Lab_6_224232_Kedrick.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 6: Naive Bayes Text Classification**
**Course:** SKM3206 – Natural Language Processing  
**Lecturer:** Assoc. Prof. Dr. Azreen Azman / Dr. Nurul Amelina Nasharuddin  
**Due date:** 21 January 2026 (Wednesday)

> ⚠️ **Instructions:**  
> 1. Go to *File → Save a copy in Drive* before editing.  
> 2. Rename your notebook as `Lab_LabNo_StudentID_Name.ipynb`.
> 3. Read the examples carefully before attempting the exercises.
> 4. Complete all the tasks in the code cells provided.  
> 5. Submit your Colab link (e.g. http://colab.research.google.com/drive/....) as a submission in the PutraBlast.  

# **Text Classification Using Naive Bayes**

---

## **1. Define the Dataset**
In this task, you are using a small dataset of messages that need to be classified as either "Spam" or "Not Spam." The dataset consists of two lists:
- **`messages`**: A list of short text messages that could be spam or not. For example, `"Free entry in a competition"` is a spam message, while `"Lunch at 1 pm?"` is a normal message.
- **`labels`**: A list of labels that classify each message. The labels are either `'Spam'` or `'Not Spam'`. These labels represent the target outcome the model will learn to predict.

For example, the first message in the list is `"Free entry in a competition"`, which is labeled as `'Spam'`. The second message `"Call me tomorrow"` is labeled as `'Not Spam'`.



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix


messages = [
    "Free entry in a competition", "Call me tomorrow",
    "Win cash prizes now", "Lunch at 1 pm?",
    "Claim your free vacation now", "Can we reschedule our meeting?",
    "Hurry, offer ends soon!"
]

labels = ['Spam', 'Not Spam', 'Spam', 'Not Spam', 'Spam', 'Not Spam', 'Spam']




## **2. Vectorizing the Text Data**
### **What is Vectorization?**
Machine learning models cannot directly work with text. We need to convert the text into numerical data. To do this, we use a technique called **vectorization**. Specifically, we use the **Bag-of-Words** model, which converts text into a matrix of word counts.

Here’s the process:
- Each unique word in the dataset is treated as a feature.
- The number of times each word appears in a message is counted.
  
For example:
- If the words in the messages are **`["Free", "entry", "competition", "Call", "tomorrow", "Win", "cash", "prizes", "Lunch", "pm"]`**, we create a feature for each of these words.
- The vectorizer counts how many times each word appears in each message and represents this as a list of numbers. For instance, the message `"Free entry in a competition"` would be represented as a vector like `[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]`, where `1` indicates the presence of a word and `0` means the word is not in the message.

This transformation results in a **bag-of-words matrix** that represents the messages numerically, making them ready for machine learning algorithms to process.




In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(messages)



## **3. Encoding Labels**
In the dataset, the labels are either `'Spam'` or `'Not Spam'`. Since machine learning algorithms work with numerical data, we need to encode these labels into numbers:
- **`'Spam'`** is encoded as `1`
- **`'Not Spam'`** is encoded as `0`

For instance:
- The label `'Spam'` would become `1`.
- The label `'Not Spam'` would become `0`.

This transformation is necessary because the model works with numbers, not text.




In [ ]:
y = [1 if label == 'Spam' else 0 for label in labels]

## **4. Splitting the Dataset**
The dataset is divided into two parts:
1. **Training Set**: This is the portion of the data used to train the model. The model will learn to recognize patterns in this data.
2. **Test Set**: This portion is used to evaluate the performance of the model. It tests how well the model can generalize to unseen data.

For instance:
- **Training Set**: The model will learn from messages like `"Free entry in a competition"`, `"Win cash prizes now"`, and `"Lunch at 1 pm?"`.
- **Test Set**: The model will test its predictions on new messages, like `"Call me tomorrow"`.

The dataset is typically split into an 80/20 or 70/30 ratio (training to test), where the majority of the data is used for training.




In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## **5. Training the Model**
Now that the data is ready, we use the **Naive Bayes** algorithm to train the model. The Naive Bayes classifier is a simple probabilistic model based on Bayes' Theorem. It works well for text classification problems because it assumes that the presence of each word is independent of the others.

### **Training Process:**
- The model is provided with the training data (messages and their corresponding labels).
- The Naive Bayes classifier learns the relationship between the words and the labels. For example, it will learn that words like `"Free"`, `"Win"`, and `"Prizes"` are often associated with spam messages.




In [ ]:

nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

## **6. Predicting on Test Data**
Once the model has been trained, we can use it to make predictions on new, unseen data. The trained model is tested on the test set (messages it has not seen before) to predict whether each message is spam or not.

For example:
- The model might predict that the message `"Call me tomorrow"` is **Not Spam** based on the learned patterns.





In [ ]:

y_pred = nb_classifier.predict(X_test)


## **7. Evaluating the Model**
To evaluate the performance of the model, we use two key metrics:
1. **Confusion Matrix**: This is a table that helps us understand how well the model is performing. It shows:
   - **True Positives (TP)**: Correctly predicted spam messages.
   - **True Negatives (TN)**: Correctly predicted non-spam messages.
   - **False Positives (FP)**: Non-spam messages incorrectly predicted as spam.
   - **False Negatives (FN)**: Spam messages incorrectly predicted as non-spam.

   For example: [[1 0] # True Negatives [0 1]] # True Positives

2. **Accuracy**: This is the proportion of correct predictions out of all predictions. It is calculated by dividing the number of correct predictions by the total number of predictions. For example, if the model made 4 predictions and got 3 correct, the accuracy would be 75%.

These metrics help us understand how well the model is working and where it might need improvement.



In [ ]:

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

## **Task for Students**

### **Question:**
Search for a text classification dataset on [Kaggle](https://www.kaggle.com/datasets/team-ai/spam-text-message-classification). Choose a dataset that involves classifying text into categories (e.g., spam vs. non-spam, sentiment analysis, etc.).

1. Download the dataset and load it into your Python environment.
2. Preprocess the dataset by:
   - Tokenizing and vectorizing the text data (e.g., using `CountVectorizer` or `TfidfVectorizer`).
   - Encoding the labels into numerical values if necessary (e.g., converting 'spam' to 1 and 'not spam' to 0).
3. Split the dataset into training and testing sets.
4. Use the **Naive Bayes** algorithm (e.g., `MultinomialNB`) to train the model on the training data.
5. Make predictions on the test data and evaluate the model by:
   - Displaying the confusion matrix.
   - Calculating the accuracy score.
6. Experiment with different preprocessing techniques or datasets and compare the results.


---
## 📤 **Submission Reminder:**  

Submit the following via PutraBlast:

- The link to your Colab notebook (as *Viewer link*). Ensure the file name is `LabNo_StudentID_Name.ipynb`.
- Link to Kaggle dataset, the confusion matrices and accuracy scores.
